# <img style="float: left; padding-right: 10px; width: 45px" src="https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/iacs.png"> CS109A Introduction to Data Science: 
## Project - 20118 FIFA World Cup



**Harvard University**<br/>
**Fall 2018**<br/>
**Instructors**: Pavlos Protopapas, Kevin Rader

<hr style="height:2pt">

In [3]:
#RUN THIS CELL 
import requests
from IPython.core.display import HTML
styles = requests.get("https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/cs109.css").text
HTML(styles)

In [4]:
import warnings
#warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import LassoCV
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import KFold

import statsmodels.api as sm
from statsmodels.regression.linear_model import OLS

from pandas.core import datetools
%matplotlib inline

/Users/paul/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:24: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.


Configuration stuff:

In [5]:
pd.options.display.max_columns = None



# EDA

In this homework, we will focus on regularization and cross validation. We will continue to build regression models for the [Capital Bikeshare program](https://www.capitalbikeshare.com) in Washington D.C.  See homework 3 for more information about the Capital Bikeshare data that we'll be using extensively. 



<div class='exercise'> <b> EDA 1 - First look </b> </div>

### Review Files / Notes

**To do**:

** A - Milesone 2:**

**1.1 Project Statement**
  - State well defined question that will be addressed in the project

**1.2 Preliminary EDA**
 - Plans
    * Review task status
    * Plans for prelimimary data exploration
    * Assign team members to tasks if need be


** B - Other Stuff**
**Logistics**
 - What tools should we use to collaborate, puts files, etc? Git?  Haven't used before but can learn.
 - Look at earlier lectures at various steps.  See who is interested in which steps?
 - How often can we meet? When can we meet? When should we meet with the TF?
 - Meeting agenda:
     * Review task status
     * Review findings
     * Discuss new tasks
     * Discuss TF meeting agenda
 
** C - Questions**

 - General
     * Milestone 2: High level goal 2: use FIFA rankings and a somewhat limited number of additional features to create a model. --> How limited?
     * Milestone 3: Asks for a baseline model.  Project goals seem to include 2 models, one that uses FIFA rankings, and one that doesn't.  Are baseline models needed for both?
     * Advice on timeline: 
         1. Construct data set (collect data)
         2. Create model based on FIFA rankings
         3. Build model from scratch with data collected
         4. Compare results from both models
         
         Organization:
             * Milestones 2-3: 5 weeks
             * Milestones 3-4: 2 weeks
             * How much homework is there going to be during this period?
 
 - Specific
     * Variable descriptions: where can we find definitions
     
     
 ** Random thoughts**
  - Could incorporate number of previous world cup appearances in the list of features
  - May need to get historical roster data
  - How does one predict a match outcome, based on a score?

In [37]:
df1 = pd.read_csv("worldcup2018/datasets/fifa/international_results.csv")

df1_sub = df1[df1["date"] >= "2016-06-01"][df1["tournament"] != "Friendly"]

df1_sub

#df1_sub.shape

#df1

/Users/paul/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until


,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
37745,2016-06-01,Curaçao,Guyana,5,2,CFU Caribbean Cup qualification,Willemstad,Curaçao,False
37749,2016-06-01,Grenada,Puerto Rico,3,3,CFU Caribbean Cup qualification,St. George's,Grenada,False
37750,2016-06-01,Latvia,Lithuania,2,1,Baltic Cup,Liepāja,Latvia,False
37751,2016-06-01,Martinique,Guadeloupe,2,0,CFU Caribbean Cup qualification,Fort-de-France,Martinique,False
37752,2016-06-01,New Caledonia,Samoa,7,0,Oceania Nations Cup,Port Moresby,Papua New Guinea,True
37754,2016-06-01,Papua New Guinea,Tahiti,2,2,Oceania Nations Cup,Port Moresby,Papua New Guinea,False
37757,2016-06-01,St. Kitts and Nevis,Suriname,1,0,CFU Caribbean Cup qualification,Basseterre,St. Kitts and Nevis,False
37758,2016-06-01,United Koreans in Japan,Northern Cyprus,1,1,ConIFA World Football Cup,Suhkumi,Georgia,True
37759,2016-06-01,Abkhazia,Sápmi,2,0,ConIFA World Football Cup,Suhkumi,Georgia,False
37761,2016-06-02,Laos,India,0,1,AFC Asian Cup qualification,Vientiane,Laos,False


In [22]:
df2 = pd.read_csv("worldcup2018/datasets/sofifa/CompleteDataset.csv")
df2.head(30)
#df2.shape

/Users/paul/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (23,35) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,Club Logo,Value,Wage,Special,Acceleration,Aggression,Agility,Balance,Ball control,Composure,Crossing,Curve,Dribbling,Finishing,Free kick accuracy,GK diving,GK handling,GK kicking,GK positioning,GK reflexes,Heading accuracy,Interceptions,Jumping,Long passing,Long shots,Marking,Penalties,Positioning,Reactions,Short passing,Shot power,Sliding tackle,Sprint speed,Stamina,Standing tackle,Strength,Vision,Volleys,CAM,CB,CDM,CF,CM,ID,LAM,LB,LCB,LCM,LDM,LF,LM,LS,LW,LWB,Preferred Positions,RAM,RB,RCB,RCM,RDM,RF,RM,RS,RW,RWB,ST
0,0,Cristiano Ronaldo,32,https://cdn.sofifa.org/48/18/players/20801.png,Portugal,https://cdn.sofifa.org/flags/38.png,94,94,Real Madrid CF,https://cdn.sofifa.org/24/18/teams/243.png,€95.5M,€565K,2228,89,63,89,63,93,95,85,81,91,94,76,7,11,15,14,11,88,29,95,77,92,22,85,95,96,83,94,23,91,92,31,80,85,88,89.0,53.0,62.0,91.0,82.0,20801,89.0,61.0,53.0,82.0,62.0,91.0,89.0,92.0,91.0,66.0,ST LW,89.0,61.0,53.0,82.0,62.0,91.0,89.0,92.0,91.0,66.0,92.0
1,1,L. Messi,30,https://cdn.sofifa.org/48/18/players/158023.png,Argentina,https://cdn.sofifa.org/flags/52.png,93,93,FC Barcelona,https://cdn.sofifa.org/24/18/teams/241.png,€105M,€565K,2154,92,48,90,95,95,96,77,89,97,95,90,6,11,15,14,8,71,22,68,87,88,13,74,93,95,88,85,26,87,73,28,59,90,85,92.0,45.0,59.0,92.0,84.0,158023,92.0,57.0,45.0,84.0,59.0,92.0,90.0,88.0,91.0,62.0,RW,92.0,57.0,45.0,84.0,59.0,92.0,90.0,88.0,91.0,62.0,88.0
2,2,Neymar,25,https://cdn.sofifa.org/48/18/players/190871.png,Brazil,https://cdn.sofifa.org/flags/54.png,92,94,Paris Saint-Germain,https://cdn.sofifa.org/24/18/teams/73.png,€123M,€280K,2100,94,56,96,82,95,92,75,81,96,89,84,9,9,15,15,11,62,36,61,75,77,21,81,90,88,81,80,33,90,78,24,53,80,83,88.0,46.0,59.0,88.0,79.0,190871,88.0,59.0,46.0,79.0,59.0,88.0,87.0,84.0,89.0,64.0,LW,88.0,59.0,46.0,79.0,59.0,88.0,87.0,84.0,89.0,64.0,84.0
3,3,L. Suárez,30,https://cdn.sofifa.org/48/18/players/176580.png,Uruguay,https://cdn.sofifa.org/flags/60.png,92,92,FC Barcelona,https://cdn.sofifa.org/24/18/teams/241.png,€97M,€510K,2291,88,78,86,60,91,83,77,86,86,94,84,27,25,31,33,37,77,41,69,64,86,30,85,92,93,83,87,38,77,89,45,80,84,88,87.0,58.0,65.0,88.0,80.0,176580,87.0,64.0,58.0,80.0,65.0,88.0,85.0,88.0,87.0,68.0,ST,87.0,64.0,58.0,80.0,65.0,88.0,85.0,88.0,87.0,68.0,88.0
4,4,M. Neuer,31,https://cdn.sofifa.org/48/18/players/167495.png,Germany,https://cdn.sofifa.org/flags/21.png,92,92,FC Bayern Munich,https://cdn.sofifa.org/24/18/teams/21.png,€61M,€230K,1493,58,29,52,35,48,70,15,14,30,13,11,91,90,95,91,89,25,30,78,59,16,10,47,12,85,55,25,11,61,44,10,83,70,11,NaN,NaN,NaN,NaN,NaN,167495,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,R. Lewandowski,28,https://cdn.sofifa.org/48/18/players/188545.png,Poland,https://cdn.sofifa.org/flags/37.png,91,91,FC Bayern Munich,https://cdn.sofifa.org/24/18/teams/21.png,€92M,€355K,2143,79,80,78,80,89,87,62,77,85,91,84,15,6,12,8,10,85,39,84,65,83,25,81,91,91,83,88,19,83,79,42,84,78,87,84.0,57.0,62.0,87.0,78.0,188545,84.0,58.0,57.0,78.0,62.0,87.0,82.0,88.0,84.0,61.0,ST,84.0,58.0,57.0,78.0,62.0,87.0,82.0,88.0,84.0,61.0,88.0
6,6,De Gea,26,https://cdn.sofifa.org/48/18/players/193080.png,Spain,https://cdn.sofifa.org/flags/45.png,90,92,Manchester United,https://cdn.sofifa.org/24/18/teams/11.png,€64.5M,€215K,1458,57,38,60,43,42,64,17,21,18,13,19,90,85,87,86,90,21,30,67,51,12,13,40,12,88,50,31,13,58,40,21,64,68,13,NaN,NaN,NaN,NaN,NaN,193080,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,7,E. Hazard,26,https://cdn.sofifa.org/48/18/players/183277.png,Belgium,https://cdn.sofifa.org/flags/7.png,90,91,Chelsea,https://cdn.sofifa.org/24/18/teams/5.png,€90.5M,€295K,2096,93,54,93,91,92,87,80,82,93,83,79,11,12,6,8,8,57,41,59,81,82,25,86,85,85,86,79,22,87,79,27,65,86,79,88.0,47.0,61.0,87.0,81.0,183277,88.0,59.0,47.0,81.0,61.0,87.0,87.0,82.0,88.0,64.0,LW,88.0,59.0,47.0,81.0,61.0,87.0,87.0,82.0,88.0,64.0,82.0
8,8,T. Kroos,27,h

In [23]:
df3 = pd.read_csv("worldcup2018/datasets/sofifa/PlayerAttributeData.csv")

df3.head(30)
#df3.shape

/Users/paul/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (11,24) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,Acceleration,Aggression,Agility,Balance,Ball control,Composure,Crossing,Curve,Dribbling,Finishing,Free kick accuracy,GK diving,GK handling,GK kicking,GK positioning,GK reflexes,Heading accuracy,ID,Interceptions,Jumping,Long passing,Long shots,Marking,Penalties,Positioning,Reactions,Short passing,Shot power,Sliding tackle,Sprint speed,Stamina,Standing tackle,Strength,Vision,Volleys
0,0,89,63,89,63,93,95,85,81,91,94,76,7,11,15,14,11,88,20801,29,95,77,92,22,85,95,96,83,94,23,91,92,31,80,85,88
1,1,92,48,90,95,95,96,77,89,97,95,90,6,11,15,14,8,71,158023,22,68,87,88,13,74,93,95,88,85,26,87,73,28,59,90,85
2,2,94,56,96,82,95,92,75,81,96,89,84,9,9,15,15,11,62,190871,36,61,75,77,21,81,90,88,81,80,33,90,78,24,53,80,83
3,3,88,78,86,60,91,83,77,86,86,94,84,27,25,31,33,37,77,176580,41,69,64,86,30,85,92,93,83,87,38,77,89,45,80,84,88
4,4,58,29,52,35,48,70,15,14,30,13,11,91,90,95,91,89,25,167495,30,78,59,16,10,47,12,85,55,25,11,61,44,10,83,70,11
5,5,79,80,78,80,89,87,62,77,85,91,84,15,6,12,8,10,85,188545,39,84,65,83,25,81,91,91,83,88,19,83,79,42,84,78,87
6,6,57,38,60,43,42,64,17,21,18,13,19,90,85,87,86,90,21,193080,30,67,51,12,13,40,12,88,50,31,13,58,40,21,64,68,13
7,7,93,54,93,91,92,87,80,82,93,83,79,11,12,6,8,8,57,183277,41,59,81,82,25,86,85,85,86,79,22,87,79,27,65,86,79
8,8,60,60,71,69,89,85,85,85,79,76,84,10,11,13,7,10,54,182521,85,32,93,90,63,73,79,86,90,87,69,52,77,82,74,88,82
9,9,78,50,75,69,85,86,68,74,84,91,62,5,12,7,5,10,86,167664,20,79,59,82,12,70,92,88,75,88,18,80,72,22,85,70,88


In [20]:
df4 = pd.read_csv("worldcup2018/datasets/sofifa/PlayerPersonalData.csv")

df4[df4["Nationality"] == "Portugal"]

#df4.head(30)
#df4.shape

,Unnamed: 0,Unnamed: 0.1,ID,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,Club Logo,Value,Wage,Special
0,0,0,20801,Cristiano Ronaldo,32,https://cdn.sofifa.org/48/18/players/20801.png,Portugal,https://cdn.sofifa.org/flags/38.png,94,94,Real Madrid CF,https://cdn.sofifa.org/24/18/teams/243.png,€95.5M,€565K,2228
67,67,67,120533,Pepe,34,https://cdn.sofifa.org/48/18/players/120533.png,Portugal,https://cdn.sofifa.org/flags/38.png,86,86,Beşiktaş JK,https://cdn.sofifa.org/24/18/teams/327.png,€13.5M,€94K,1839
94,94,94,218667,Bernardo Silva,22,https://cdn.sofifa.org/48/18/players/218667.png,Portugal,https://cdn.sofifa.org/flags/38.png,84,91,Manchester City,https://cdn.sofifa.org/24/18/teams/10.png,€43.5M,€165K,2012
133,133,133,212814,João Mário,24,https://cdn.sofifa.org/48/18/players/212814.png,Portugal,https://cdn.sofifa.org/flags/38.png,83,88,Inter,https://cdn.sofifa.org/24/18/teams/44.png,€34M,€70K,2098
141,141,141,207566,William Carvalho,25,https://cdn.sofifa.org/48/18/players/207566.png,Portugal,https://cdn.sofifa.org/flags/38.png,83,86,Sporting CP,https://cdn.sofifa.org/24/18/teams/237.png,€27.5M,€21K,1967
146,146,146,200888,Danilo Pereira,25,https://cdn.sofifa.org/48/18/players/200888.png,Portugal,https://cdn.sofifa.org/flags/38.png,83,86,FC Porto,https://cdn.sofifa.org/24/18/teams/236.png,€27.5M,€18K,2029
148,148,148,199482,A. Lopes,26,https://cdn.sofifa.org/48/18/players/199482.png,Portugal,https://cdn.sofifa.org/flags/38.png,83,86,Olympique Lyonnais,https://cdn.sofifa.org/24/18/teams/66.png,€22.5M,€60K,1317
167,167,167,184826,Adrien Silva,28,https://cdn.sofifa.org/48/18/players/184826.png,Portugal,https://cdn.sofifa.org/flags/38.png,83,83,NaN,https://cdn.sofifa.org/flags/38.png,€0,€0,2174
174,174,174,178005,Rui Patrício,29,https://cdn.sofifa.org/48/18/players/178005.png,Portugal,https://cdn.sofifa.org/flags/38.png,83,83,Sporting CP,https://cdn.sofifa.org/24/18/teams/237.png,€17.5M,€19K,1350
192,192,192,139068,Nani,30,https://cdn.sofifa.org/48/18/players/139068.png,Portugal,https://cdn.sofifa.org/flags/38.png,83,83,Lazio,https://cdn.sofifa.org/24/18/teams/46.png,€24.5M,€47K,1949


In [10]:
df5 = pd.read_csv("worldcup2018/datasets/sofifa/PlayerPlayingPositionData.csv")

df5.head(30)
#df5.shape

,Unnamed: 0,CAM,CB,CDM,CF,CM,ID,LAM,LB,LCB,LCM,LDM,LF,LM,LS,LW,LWB,Preferred Positions,RAM,RB,RCB,RCM,RDM,RF,RM,RS,RW,RWB,ST
0,0,89.0,53.0,62.0,91.0,82.0,20801,89.0,61.0,53.0,82.0,62.0,91.0,89.0,92.0,91.0,66.0,ST LW,89.0,61.0,53.0,82.0,62.0,91.0,89.0,92.0,91.0,66.0,92.0
1,1,92.0,45.0,59.0,92.0,84.0,158023,92.0,57.0,45.0,84.0,59.0,92.0,90.0,88.0,91.0,62.0,RW,92.0,57.0,45.0,84.0,59.0,92.0,90.0,88.0,91.0,62.0,88.0
2,2,88.0,46.0,59.0,88.0,79.0,190871,88.0,59.0,46.0,79.0,59.0,88.0,87.0,84.0,89.0,64.0,LW,88.0,59.0,46.0,79.0,59.0,88.0,87.0,84.0,89.0,64.0,84.0
3,3,87.0,58.0,65.0,88.0,80.0,176580,87.0,64.0,58.0,80.0,65.0,88.0,85.0,88.0,87.0,68.0,ST,87.0,64.0,58.0,80.0,65.0,88.0,85.0,88.0,87.0,68.0,88.0
4,4,NaN,NaN,NaN,NaN,NaN,167495,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,84.0,57.0,62.0,87.0,78.0,188545,84.0,58.0,57.0,78.0,62.0,87.0,82.0,88.0,84.0,61.0,ST,84.0,58.0,57.0,78.0,62.0,87.0,82.0,88.0,84.0,61.0,88.0
6,6,NaN,NaN,NaN,NaN,NaN,193080,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,7,88.0,47.0,61.0,87.0,81.0,183277,88.0,59.0,47.0,81.0,61.0,87.0,87.0,82.0,88.0,64.0,LW,88.0,59.0,47.0,81.0,61.0,87.0,87.0,82.0,88.0,64.0,82.0
8,8,83.0,72.0,82.0,81.0,87.0,182521,83.0,76.0,72.0,87.0,82.0,81.0,81.0,77.0,80.0,78.0,CDM CM,83.0,76.0,72.0,87.0,82.0,81.0,81.0,77.0,80.0,78.0,77.0
9,9,81.0,46.0,52.0,84.0,71.0,167664,81.0,51.0,46.0,71.0,52.0,84.0,79.0,87.0,82.0,55.0,ST,81.0,51.0,46.0,71.0,52.0,84.0,79.0,87.0,82.0,55.0,87.0
